<a href="https://colab.research.google.com/github/Nacho2904/orga_de_datos/blob/main/tp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico II: Spark

## Setup

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 51.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=a0c9eccd04320e480ef21880ed9c969565c3df7c19e27337574b85eefe7b2f76
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive 

drive.mount('/content/gdrive')
path_a_cast = 'gdrive/MyDrive/Parquets/cast.parquet'
path_a_companies = 'gdrive/MyDrive/Parquets/companies.parquet'
path_a_crew = 'gdrive/MyDrive/Parquets/crew.parquet'
path_a_actores = 'gdrive/MyDrive/Parquets/imdb_actors.parquet'
path_a_keywords = 'gdrive/MyDrive/Parquets/keywords.parquet'
path_a_links = 'gdrive/MyDrive/Parquets/links.parquet'
path_a_movies = 'gdrive/MyDrive/Parquets/movies.parquet'
path_a_ratings = 'gdrive/MyDrive/Parquets/ratings.parquet'


Mounted at /content/gdrive


In [4]:
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


## S7

¿Cuál fue el año más mortal para los actores y actrices?

In [ ]:
AÑO_FALLECIMIENTO = 3
CONTEO_MUERTES = 1
rdd_actores = sqlContext.read.parquet(path_a_actores).rdd

In [ ]:
rdd_años_de_fallecimiento = rdd_actores.map(lambda row: row[AÑO_FALLECIMIENTO])
rdd_años_de_fallecimiento.first()

'1987'

In [ ]:
rdd_años_de_fallecimiento = rdd_años_de_fallecimiento.filter(str.isnumeric)
rdd_años_de_fallecimiento = rdd_años_de_fallecimiento.map(int)
dict_con_conteo = dict(rdd_años_de_fallecimiento.countByValue())
año_mas_mortal = sorted([[llave, dict_con_conteo[llave]] for llave in dict_con_conteo], 
                        key = lambda año_y_conteo: año_y_conteo[CONTEO_MUERTES])[-1]
print(año_mas_mortal)

[2021, 3409]


## S23

¿Cuál es el usuario más pesimista (menor rating promedio) para cada género de películas? Considere solo usuarios con más de 20 reviews

In [ ]:
rdd_ratings = sqlContext.read.parquet(path_a_ratings).rdd
rdd_ratings.take(5)

[Row(userId=1, movieId=110, rating=1.0, timestamp=1425941529),
 Row(userId=1, movieId=147, rating=4.5, timestamp=1425942435),
 Row(userId=1, movieId=858, rating=5.0, timestamp=1425941523),
 Row(userId=1, movieId=1221, rating=5.0, timestamp=1425941546),
 Row(userId=1, movieId=1246, rating=5.0, timestamp=1425941556)]

In [ ]:
rdd_users_with_more_than_20_reviews = rdd_ratings.map(lambda rating: (rating[0], 1)).countByKey().filter(lambda user: user[1] > 20)

In [ ]:
rdd_ratings_as_key_value = rdd_ratings.map(lambda rating: (rating[0], rating[1:]))
rdd_ratings_filtered = rdd_ratings_as_key_value.join(rdd_users_with_more_than_20_reviews).map(
    lambda row: (np.int64(row[1][0][0]), (row[0], row[1][0][1])))

movieId, (userId, rating)

=> movieId, ((userId, rating), listOfGenres) 

=> userId, (rating, listOfGenres, 1)

In [ ]:
rdd_movies = sqlContext.read.parquet(path_a_movies).rdd
rdd_movies_id_and_genre = rdd_movies.map(lambda movie: (movie[5], movie[3])).filter(lambda row: row[1]).map(
    lambda movie: (np.int64(movie[0]), movie[1].split(',')))
rdd_ratings_filtered_with_genres = rdd_ratings_filtered.join(rdd_movies_id_and_genre)
rdd_ratings_filtered_with_genres = rdd_ratings_filtered_with_genres.map(lambda row: (row[1][0][0], (row[1][0][1], row[1][1])))

userId, (rating, listOfGenres)

=> userId, (genresDict)

genresDict <- {genre: [ratingsOfGenre] for genre in genres}

In [ ]:
rdd_grouped_by_user = rdd_ratings_filtered_with_genres.groupByKey().map(lambda row: (row[0], list(row[1])))

acá en algún punto debería copiar y pegar la forma en la que saco todos los géneros

In [ ]:
def get_mean_ratings_per_genre(ratings_and_genres):
  all_genres = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
  'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
   'TV Movie', 'Thriller', 'War', 'Western']

  ratings = {genre: [] for genre in all_genres}
  for (rating, genres) in ratings_and_genres:
    for genre in genres:
      ratings[genre].append(rating)
  return {genre: np.sum(np.array(ratings[genre]))/len(ratings[genre]) if ratings[genre] else np.inf for genre in all_genres}

rdd_grouped_by_user_mean_ratings = rdd_grouped_by_user.map(lambda user: (user[0], get_mean_ratings_per_genre(user[1])))

In [ ]:
genres = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary', 'Drama',
  'Family', 'Fantasy', 'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction',
   'TV Movie', 'Thriller', 'War', 'Western']

for genre in genres:
  most_pesimistic_user = rdd_grouped_by_user_mean_ratings.reduce(
      lambda user_1, user_2: user_1 if user_1[1][genre] < user_2[1][genre] else user_2)[0]
  print(f"El usuario más pesimista con películas de {genre} es {most_pesimistic_user}")

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
"""
dejo los resultados acá para no tener que correrlo de vuelta
El usuario más pesimista con películas de Action es 232007
El usuario más pesimista con películas de Adventure es 232007
El usuario más pesimista con películas de Animation es 158483
El usuario más pesimista con películas de Comedy es 232007
El usuario más pesimista con películas de Crime es 252245
El usuario más pesimista con películas de Documentary es 219341
El usuario más pesimista con películas de Drama es 76193
El usuario más pesimista con películas de Family es 179759
El usuario más pesimista con películas de Fantasy es 205283
El usuario más pesimista con películas de Foreign es 23789
El usuario más pesimista con películas de History es 154619
El usuario más pesimista con películas de Horror es 205283
El usuario más pesimista con películas de Music es 93275
El usuario más pesimista con películas de Mystery es 222785
El usuario más pesimista con películas de Romance es 93275
El usuario más pesimista con películas de Science Fiction es 76193
El usuario más pesimista con películas de TV Movie es 21071
El usuario más pesimista con películas de Thriller es 76193
El usuario más pesimista con películas de War es 191147
El usuario más pesimista con películas de Western es 154247
"""

## S25

¿Cuál es el percentil 95 de la popularidad?

In [29]:
POPULARITY = 10
rdd_movies = sqlContext.read.parquet(path_a_movies).rdd

rdd_popularities = rdd_movies.map(lambda row: row[POPULARITY]).filter(lambda row: row)
number_of_movies = rdd_popularities.count()
top_5_per_cent = rdd_popularities.takeOrdered(int(number_of_movies*0.05), key = lambda x: -x)
percentile_95th = top_5_per_cent[-1]
print(percentile_95th)

11.063822


## S11

 Según los cinéfilos, ¿cuál es el rating promedio de las 5 películas más populares? Un usuario es cinéfilo cuando puntuó más de 50 películas y todas las puntuaciones son de más de 2.5 estrellas. Indicar id, título, popularidad y rating promedio de la película.

In [ ]:
rdd_ratings = sqlContext.read.parquet(path_a_ratings).rdd
rdd_movies = sqlContext.read.parquet(path_a_movies).rdd

In [ ]:
rdd_ratings_key_value = rdd_ratings.map(lambda row: (row[0], [row[1], row[2]]))

In [ ]:
user_has_no_ratings_below = lambda user, threshold: all([rating[1] > threshold for rating in user[1]]) 
user_has_more_than_25_ratings = lambda user: len(user[1]) > 25
rdd_ratings_grouped_per_user = rdd_ratings_key_value.groupByKey().map(lambda row: (row[0], list(row[1])))
rdd_usuarios_cinefilos = rdd_ratings_grouped_per_user.filter(lambda user: user_has_no_ratings_below(user, 2.5) and 
                                                         user_has_more_than_25_ratings(user)).map(lambda user: user[0])

In [ ]:
rdd_ratings_key_value_cinefilos = rdd_ratings_key_value.join(rdd_usuarios_cinefilos)

In [ ]:
rdd_ratings_key_value_cinefilos.take(1)

[(8020, [168, 4.0])]

In [ ]:
popularity = 10
rdd_top_5_most_popular_movies = rdd_movies.filter(lambda row: row[10])
rdd_top_5_most_popular_movies = rdd_top_5_most_popular_movies.takeOrdered(5, key = lambda row: -row[popularity])

In [ ]:
rdd_top_5_most_popular_movies[0][10]

547.488298

## S34

 ¿Cuál es la probabilidad de que una película de un género en particular tenga una calificación promedio mayor a 3 (utilizando la tabla de ratings)? Obtenga las probabilidades de cada género posible.

In [30]:
rdd_ratings = sqlContext.read.parquet(path_a_ratings).rdd.cache()
rdd_movies = sqlContext.read.parquet(path_a_movies).rdd

In [51]:
rdd_movies_genres = rdd_movies.map(lambda row: set(row[3].split(',')) if row[3] else {})
genres = rdd_movies_genres.reduce(lambda genres_1, genres_2: genres_1.union(genres_2))
genres

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}